# <h1> Project 1

# <h2> Gradient - based optimization and differentiable programming

# <h2> Pole - Cart Balancing Controller Design with Neural Network and Gradient Descent.

# <h3> Problem Definition

In this problem, the aim is to design a controller $K$ that makes the open-loop pole cart problem into a closed loop system.
<br/> The controller $K$ helps the pole - cart system to reach desired cart position $r$ and the pole deviation to $0\;degrees$.
<br/> Designing the parameters of $K$ can be tricky and difficult.
<br/> Some common methods of designing the controller are PID and LQR based controllers.
<br/> In this problem, we design controller $K$ using Neural Network and Gradient Descent such that the cart reaches $0.4\;m$ and
the pole stays upright at $0\;degrees$

In [200]:
# Importing Libraries
import logging
import numpy as np
import torch as t
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
logger = logging.getLogger(__name__)

# <h3> System Properties

The pole - cart system is clearly described in the diagram below:<br/>
![Pole](/assets/images/Cart_Pendulum.PNG "Pole-Cart")

In order to solve the problem, the states needs to be defined accurately. The states chosen are $x = [x\;\dot{x}\;\theta\;\dot{\theta}]$, where
$x$ is the cart position in meters and $\theta$ is the pole deviation from vertical in $rads$.
![State_Space!](/assets/images/State_Space.PNG "State Space")
<br/> Where the parameters are as follows:<br/>
1. Mass of Cart ($M$) = 0.5 $kg$
2. Mass of Pole ($m$) = 0.2 $kg$
3. Co-effienct of Friction ($b$) = 0.1 $N/m/sec$
4. Length to Pole Center of Mass ($l$) = 0.3 $m$
5. Mass Moment of Inertia of Pole = 0.006 $kgm^2$
<br/>Since, the states of interest are $x$ and $\theta$ the $C$ matrix is given as:<br/>
$C = \begin{bmatrix}
1&0&0&0\\
0&0&1&0
\end{bmatrix}
$
<br/> The state transition matrix $A$ and control matrix $B$ in discrete states are derived from
continuous state-space model presented by **[UMichigan CTMS](https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=ControlDigital)**.

In [201]:
# Setting A, B, C and D matrices
A = t.tensor([[1,0.009991,0.0001336,4.453e-07],
             [0,0.9982,0.02672,0.0001336],
             [0,-2.272e-05,1.002,0.01001],
             [0,-0.004544,0.3119,1.002]]).float()
B = t.tensor([[9.086e-05],
              [0.01817],
              [0.0002272],
              [0.04544]]).float()

To create the State Transition loop, we need to understand the control diagram below (courtesy: **[UMichigan CTMS](https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=ControlDigital)**):
![Control Diagram with Feedback!](/assets/images/Control%20Diagram.PNG "Control Diagram Discrete")
<br/> Based on the diagram the loop becomes:<br/>
**Measurement Update:**<br/>
$y = Cx$<br/>
**State Update:**<br/>
$u = r - Kx$<br/>
$x_{n+1} = Ax_{n} + Bu$
<br/> This is programmed in the class $Dynamics$ that will simulate the state transition.

# <h3> Defining System Dynamics

In [202]:
# Defining State Transiton Dynamics
class Dynamics(nn.Module):
    def __init__(self):
        super(Dynamics, self).__init__()
    @staticmethod
    def forward(state,k,ref):
        kx = t.tensor([ref]) - t.matmul(k.reshape(1,4),state.reshape(4,1)) #Calculating u
        dx = t.matmul(B,kx).reshape(4,1) #Calculating Bu
        state = t.matmul(A,state.reshape(4,1)) + dx #Calculating x(n+1)
        return state.reshape(1,4)

# <h3> Controller Neural Network

The controller $K$ is assumed to be some function of states $x$ such that $K = f(x)$ that minimizes error
between $y$ and $r$. To do that, the neural network is chosen of the following architecture.<br/>
**Input Dimension: 4**<br/>
**Activation Function: Tanh**<br/>
**Hidden Layer Dimension: 10**<br/>
**Activation Function: Tanh**<br/>
**Hidden Layer Dimension: 10**<br/>
**Ouput Layer Dimension: 4**<br/>
This makes sure that the neural network predicts the $4$ gains in the controller $K_{1\times4}$ .

In [203]:
# Defining Controller K that is supposed to help the Cart and Pole reach the reference positions
class Controller(nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input,dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden,dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden,dim_output)
        )

    def forward(self, state):
        k = self.network(state)
        return k

# <h3> Simulation Class

Here, the $Simulation.forward()$ function simulates the states from $t_0$ to $t_{final}$ with increments of $\delta T$.
<br/> $t_0 = 0$, $t_{final} = 2\;seconds$ and $\delta T = 0.01$.
<br/> The error is chosen as Least Squares defined as:<br/>
$min_{error} = \sum (x_{1}-r)^2 + x_{3}^2$,<br/>
where $x_1$ is cart position, $r$ is desired position of cart and $x_3$ is pole deviation from vertical.

In [204]:
# Define Simulator that calculates new states
class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T,ref):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.ref = ref
        self.optim = optim.SGD(self.controller.parameters(),lr=0.1)
        self.action_trajectory = []

    def forward(self, state):
        self.state_trajectory = []
        #loss = t.zeros(size=(T,1))
        state = self.state
        action = self.controller.forward(state)
        for i in range(T):
            self.state_trajectory.append(state)
            state = self.dynamics.forward(state,action,ref)
            loss = self.error(state,ref)
            self.optim.zero_grad()
            loss.backward(retain_graph = True)
        self.action_trajectory.append(action)
        return loss

    @staticmethod
    def initialize_state():
        state = [0.,0.,0.,0.]
        return t.tensor(state,requires_grad=False).float()
    def error(self,state,ref):
        return (state[0][0]-ref)**2 + state[0][2]**2

# <h3> Optimize Class

In [205]:
class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.LBFGS(self.parameters, lr=0.01)# Optimizer Function
    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state) # Calculate Loss
            self.optimizer.zero_grad() # Clear Gradient for next iteration
            loss.backward() # Back Propagate and Update weights.
            return loss
        self.optimizer.step(closure)
        return closure()


    def train(self, epochs):
        loss_vec = [] # Store loss to plot convergence
        for epoch in range(epochs):
            #loss = self.step()
            loss = self.simulation(self.simulation.state)
            loss_vec.append(loss)
            print('[%d] loss: %.3f' % (epoch + 1, loss))
            #if epoch%5==0:
               #self.visualize() #Plot States every 5 epochs
        return loss_vec

    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        x = data[:,0,0]
        y = data[:,0,2]
        fig,ax = plt.subplots()
        ax1 = ax.twinx()
        ax.plot(x,color = 'tab:red',label='Cart')
        ax.set_ylabel('Meters (m)',color = 'tab:red')
        ax.set_xlabel('Time Step (seconds x 100)')
        ax.legend()
        ax1.plot(y,color = 'tab:blue',label = 'Pole')
        ax1.set_ylabel('Radians (rad)',color = 'tab:blue')
        ax1.legend()
        plt.title('Step Response')
        plt.show()

# <h3> Run system

In [206]:
# Parameters for Running the System
dim_input = 4 # Number of Inputs to Neural Network = Number of States
dim_output = 4 # Number of Outputs from Neural Network = Number of Gains of K
dim_hidden = 10 # Hidden Layers for advanced computation of gains
ref = 0.4 #Reference Position for Cart
T = 300 #Time Steps to Simulate
d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c,d, T,ref)  # define simulation
o = Optimize(s)  # define optimizer
error = o.train(100) # Store Error for Convergence Plot

[1] loss: 219357724672.000
[2] loss: 219357724672.000
[3] loss: 219357724672.000


KeyboardInterrupt: 

In [ ]:
# Plotting Convergence
plt.plot(t.tensor(error).detach().numpy())
plt.ylabel('Error')
plt.xlabel('Epochs')
plt.title('Error Convergence')
plt.show()

In [ ]:
# Ideal Controller Gains
print('The ideal Controller K = ',s.action_trajectory[-1][:].clone().detach().numpy())

# <h3> Analysis of Results
Using the Controller $K = \begin{bmatrix}
-0.0205051 & -3.553205  &1.542318  &3.27144
\end{bmatrix}$, we were able to bring the position of the cart to $r =0.4$ meters while the pole was kept upright at $0$ degrees.
<br/> The state trajectory of the system with the optimized controller can be visualized below:<br/>

In [ ]:
o.visualize()

This shows that the controller was able to stabilize the system in 2 seconds.<br/>
Thus it can be claimed that one can find an ideal controller $K$ for a dynamic system that optimizes the
required step response.<br/>
Future improvement could include extending this approach to:<br/>
1. Stabilizing both pole and cart position to a certain reference.<br/>
2. Stabilizing a dual pole system on the same cart.

In [ ]:
w= t.linalg.eigvals(A - t.matmul(B.reshape(4,1),s.action_trajectory[-1][:].reshape(1,4)))
v = (t.view_as_real(w))
v[:,:2]

In [ ]:
t.empty(size = (200,1))